## Correlate exposures with telemetry

In this notebook, we demonstrate using the LSST EFD client.

This is complementaty to the [Chronograf](https://test-chronograf-efd.lsst.codes) interface which we use for time-series visualization.

You'll need to install `pandas`, `astropy` and `matplotlib` to run this notebook.

In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pylab as plt
import pandas as pd
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

We'll access the DM-EFD instance deployed at the AuxTel lab in Tucson. You need to be on site or connected to the NOAO VPN. 

To access the EFD, you will need to put a file called `.lsst/notebook_auth.yaml` in your home directory.  It should be formatted in the following way (substituting the appropriate values, of course).  Ping Anglo or Simon on Slack (`@afausti`, `@ksk`) if you have any problem.

```yaml
test:
  username: "user"
  host: "endpoint.edu"
  password: "passwd"
```

In [ ]:
client = EfdClient('int_efd')

Specify a time range for data in `InfluxQL`.  This notebook looks at data from 30 days ago.

In [ ]:
t2 = Time('2020-03-15T12:00:00', scale='tai')
t1 = t2 - TimeDelta(24*3600, format='sec', scale='tai')

Query each of the measurements we may want to correlate later in the notebook.  Note that this could be done as a single query, but the result is a dictionary of `DataFrames` which I find less convenient to use than named variables corresponding to one `DataFrame` each.

In [ ]:
df1 = await client.select_time_series("lsst.sal.ATCamera.wreb", ["temp1", "temp2", "temp3", "temp4", "temp5", "temp6", "ccdTemp0"], t1, t2)

df2 = await client.select_time_series("lsst.sal.ATCamera.command_takeImages", ["expTime", "numImages"], t1, t2)

df3 = await client.select_time_series("lsst.sal.ATCamera.wrebPower", "analog_I", t1, t2)

shutter_close = await client.select_time_series("lsst.sal.ATCamera.logevent_endShutterClose", "priority", t1, t2)

shutter_open = await client.select_time_series("lsst.sal.ATCamera.logevent_endShutterOpen", "priority", t1, t2)

start_readout = await client.select_time_series("lsst.sal.ATCamera.logevent_startReadout", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

end_readout = await client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

start_integration = await client.select_time_series("lsst.sal.ATCamera.logevent_startIntegration", ["priority", "private_sndStamp", "private_rcvStamp"], t1, t2)

Select a telemetry stream, `analog_I`, and overplot log messages from the `start_integration`, `start_readout`, and `end_readout` log streams.

In [ ]:
plt = df3.plot(y="analog_I")
for i, r in start_integration.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='g', alpha=0.3)
for i, r in start_readout.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='b', alpha=0.3)
for i, r in end_readout.iterrows():
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='r', alpha=0.3)

Inspect time stamps.  There are three obvious ones: the timestamp from `InfluxDB`, `private_rcvStamp` from SAL, and `private_sndStamp` from SAL.

In [ ]:
plt = df3.plot()
diff = []
for i, r in start_integration.iterrows():
    plt.axvline(i, c='g', alpha=0.3)
    plt.axvline(pd.Timestamp(r['private_rcvStamp'], unit='s'), c='b', alpha=0.3)
    plt.axvline(pd.Timestamp(r['private_sndStamp'], unit='s'), c='r', alpha=0.3)
    diff.append(pd.Timestamp(r['private_rcvStamp'], unit='s') - pd.Timestamp(r['private_sndStamp'], unit='s'))
    diff[-1] = diff[-1].value/1000000000

Look at the histogram of timestamp differences for `private_rcvStamp` - `private_sndStamp`.

In [ ]:
diff_df = pd.DataFrame(diff)
diff_df.hist(bins=50)